In [112]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression

In [80]:
def gorizont_coord(file):
   with open(file, 'r') as f:
       gorizon = {}
       inlines = set()
       xlines = set()
       for line in f:
           if not line.startswith('#') and line.strip():  # Пропуск строк, начинающихся с #
               x, y, z, *rest = line.lstrip().split()  # Разбиение строки по столбцам
               x = int(x.split('.')[0])  # Получение целочисленного значения из первого столбца
               y = int(y.split('.')[0])  # Получение целочисленного значения из второго столбца
               z = int(z.split('.')[0])  # Получение целочисленного значения из третьего столбца
               #gorizon.append([x, y, z])  # Добавление данных в словарь
               gorizon[f"{x}_{y}"] = z
               inlines.add(x)
               xlines.add(y)
          
   return gorizon, list(inlines), list(xlines)

In [64]:
# !!! только для старого файла!!!  
# По словам Саши для проверки нужны были строки только с "J1_1_TOP_S". 
def otbivki_coord_0(file): 
    with open(file, 'r') as f:
        gorizon = []
        for line in f:
            if line.strip() and line[0].isdigit():
                if line.split()[9] == '"J1_1_TOP_S"':
                   x, y, z, *rest = line.split()  # Разбиение строки по столбцам
                   x = int(x.split('.')[0])  # Получение целочисленного значения из первого столбца
                   y = int(y.split('.')[0])  # Получение целочисленного значения из второго столбца
                   z = int(z.split('.')[0])  # Получение целочисленного значения из третьего столбца
                   if z >=0:
                       z = -1*z
                   gorizon.append([x,y,z])
                             
    return gorizon

In [68]:
def otbivki_coord(file): 
    with open(file, 'r') as f:
        gorizon = []
        for line in f:
            if line.strip() and line[0].isdigit():
                x, y, z, *rest = line.split()  # Разбиение строки по столбцам
                x = int(x.split('.')[0])  # Получение целочисленного значения из первого столбца
                y = int(y.split('.')[0])  # Получение целочисленного значения из второго столбца
                z = int(z.split('.')[0])  # Получение целочисленного значения из третьего столбца
                if z >=0:
                    z = -1*z
                gorizon.append([x,y,z])
          
    return gorizon

In [102]:
Top_TVDSS, inlines_TVDSS, xlines_TVDSS = gorizont_coord('C:/HV/Seismic/datas/TVDSS_Top_U1.txt')
otbivki_Petrel_well_tops_ASCII = otbivki_coord_0('C:/HV/Seismic/datas/Petrel_well_tops_ASCII.txt') #это старые отбивки
otbivki_U11_point_set_top_corrected = otbivki_coord('C:/HV/Seismic/datas/U11_point_set_top_corrected.txt')

In [103]:
def data_for_graf (list1, dict2, list3, list4):
    """
    list1 - отбивки
    dict2 - карта
    list3 - инлайны
    list4 - икслайны
    """
    X = []
    Y = []
    for i in range(len(list1)):
        inline = min(list3, key=lambda x: abs(x - list1[i][0]))
        xline = min(list4, key=lambda x: abs(x - list1[i][1]))
        X.append(dict2[f"{inline}_{xline}"])
        Y.append(list1[i][2])
    return X, Y

In [116]:

def proverka_korr_skor_model_1(list1, dict2, list3, list4):
    """
    list1 - отбивки
    dict2 - карта
    list3 - инлайны
    list4 - икслайны
    возвращает 4 переменные: уравнение, R2б верхняя и нижняя границы коридора неопределенности (в виде списков)
    """
    X, Y = data_for_graf(list1, dict2, list3, list4)
    
    X = np.array(X).reshape(-1, 1)  
    Y = np.array(Y)

    # Создание модели линейной регрессии
    model = LinearRegression()
    model.fit(X, Y)
    Y_pred = model.predict(X)
    slope = model.coef_[0]
    intercept = model.intercept_
    print(f'Уравнение регрессии: Y = {slope:.2f} * X + {intercept:.2f}')
    equation = f'Y = {slope:.2f} * X + {intercept:.2f}'
    r_squared = model.score(X, Y)
    print(f"Коэффициент детерминации R^2: {r_squared:.4f}")
    
    # Расчет стандартной ошибки и доверительного интервала
    residuals = Y - Y_pred
    std_error = np.std(residuals)
    confidence_interval = 3 * std_error
    upper_bound = Y_pred + confidence_interval
    lower_bound = Y_pred - confidence_interval
    
    # Поиск выбросов
    outliers = []
    for i in range(len(Y)):
        if Y[i] > upper_bound[i] or Y[i] < lower_bound[i]:
            outliers.append((X[i][0], Y[i]))
    
    if len(outliers) != 0:
        print('Проверьте корректность скоростной модели. Наблюдаются большие расхождения время/глубина в скважинах')
    else:
        print("тест пройден")
    
    

    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Scatter(x=X.flatten(), y=Y, mode='markers', name='Данные', marker=dict(color='blue')))
    fig.add_trace(go.Scatter(x=X.flatten(), y=Y_pred, mode='lines', name='Линейная тренда', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=X.flatten(), y=upper_bound, mode='lines', name='Верхний предел', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=X.flatten(), y=lower_bound, mode='lines', name='Нижний предел', line=dict(color='green')))
    
    if outliers:
        outlier_x, outlier_y = zip(*outliers)
        fig.add_trace(go.Scatter(x=outlier_x, y=outlier_y, mode='markers', name='Выбросы', marker=dict(color='orange', size=10)))

    fig.update_layout(title='Проверка корректности скоростной модели',
                      xaxis_title='X',
                      yaxis_title='Y',
                      showlegend=True)
    
    # Отображение графика
    fig.show()
    return equation, r_squared, upper_bound, lower_bound

In [114]:
equation, r_squared, upper_bound, lower_bound = proverka_korr_skor_model_1(otbivki_U11_point_set_top_corrected, Top_TVDSS, inlines_TVDSS, xlines_TVDSS)

Уравнение регрессии: Y = 0.97 * X + -73.71
Коэффициент детерминации R^2: 0.8508
тест пройден


In [115]:
equation_0, r_squared_0, upper_bound_0, lower_bound_0 = proverka_korr_skor_model_1(otbivki_Petrel_well_tops_ASCII, Top_TVDSS, inlines_TVDSS, xlines_TVDSS)

Уравнение регрессии: Y = 0.95 * X + -127.46
Коэффициент детерминации R^2: 0.6401
Проверьте корректность скоростной модели. Наблюдаются большие расхождения время/глубина в скважинах
